# n Player FPSB Auction with symmetric valuation distributions

## Imports

In [ ]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
import time
from timeit import default_timer as timer

import torch
import torch.nn as nn
import torch.nn.utils as ut
from torch.optim.optimizer import Optimizer, required

from bnelearn.strategy import NeuralNetStrategy, ClosureStragegy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import FirstPriceSealedBidAuction, VickreyAuction
from bnelearn.optimizer import ES
from bnelearn.environment import AuctionEnvironment

from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

# set up matplotlib
is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display
    
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'

# Use specific cuda gpu if desired 
#(i.e. for running multiple experiments in parallel)
specific_gpu = 5
if cuda and specific_gpu:
    torch.cuda.set_device(specific_gpu)

print(device)
if cuda: print(torch.cuda.current_device())

## Settings

The following cell fully defines an experiment.

The folowing settings are known to work well:
```
3p:  batch_size = 2**17, learning_rate = 5e-1, lr_decay_every = 500, lr_decay_factor = 0.7,
     momentum = 0.5, sigma = .02, n_perturbations = 128,
     eval_batch_size 2**15
     hidden_nodes = [5,5], hidden_activations = [nn.SELU(), nn.SELU()]
     
     
5p:  batch_size = 2**17, size_hidden_layer = 10, learning_rate = 5e-1, lr_decay_every = 1000, lr_decay_factor = 0.75,
     momentum = 0.7, sigma = .02, n_perturbations = 256,
     eval_batch_size 2**15
     (1 hidden layer, tanh)
     
10p: batch_size = 2**17, size_hidden_layer = 10, learning_rate = 5e-1, lr_decay_every = 1000, lr_decay_factor = 0.8,
     momentum = 0.6, sigma = .02, n_perturbations = 128,
     eval_batch_size 2**15
     (1 hidden layer, tanh)
     
```

In [ ]:
run_comment = '' # used in log title

## Experiment setup
n_players = 3
n_items = 1

# valuation distribution
valuation_mean = 10.0
valuation_std = 5.0

## Environment settings
#training batch size
batch_size = 2**17
input_length = 1

eval_batch_size = 2**15 # FAILS for 2**16... why?
n_processes_optimal_strategy = 32

# strategy model architecture
hidden_nodes = [5, 5]
hidden_activations = [nn.SELU(), nn.SELU()]

# optimization params
epoch = 5000
learning_rate = 5e-1
lr_decay = True
lr_decay_every = 500
lr_decay_factor = 0.70
baseline = True
momentum = 0.6

sigma = .02 #ES noise parameter
n_perturbations = 128

# plot and log training options
plot_epoch = 200 #plot and log optima this often
write_graph = True # whether to log graph to disk
plot_points = min(100, batch_size)
plot_xmin = int(max(0, valuation_mean - 3*valuation_std))
plot_xmax = int(valuation_mean + 3*valuation_std)

plot_ymin = 0
plot_ymax = 20

def strat_to_bidder(strategy, batch_size=batch_size, player_position=None, cache_actions=False):
    return Bidder.normal(valuation_mean,valuation_std, strategy,
                         batch_size = batch_size,
                         player_position=player_position,
                         cache_actions=cache_actions)

# Setting up the Environment

In [ ]:
mechanism = FirstPriceSealedBidAuction(cuda = True)

model = NeuralNetStrategy(input_length,
                          hidden_nodes = hidden_nodes,
                          hidden_activations = hidden_activations,
                          requires_grad=False,
                          ensure_positive_output = torch.tensor([float(valuation_mean)])
                          ).to(device) 

bidders = [strat_to_bidder(model, batch_size, player_position)
           for player_position in range(n_players)]

env = AuctionEnvironment(mechanism,
                  agents = bidders,
                  batch_size = batch_size,
                  n_players =n_players,
                  strategy_to_player_closure = strat_to_bidder
                 )
optimizer = ES(model=model, environment = env,
               lr = learning_rate, momentum=momentum,
               sigma=sigma, n_perturbations=n_perturbations)

print(model)
n_parameters = sum([p.numel() for p in model.parameters()])
print('Total parameters: ' + str(n_parameters))

## Setting up Evaluation and logging

###  Optimal Bid Function

According to Menezes et al. 2005., the optimal bid for symmetric valuations $v$ that are distributed with cdf $F(v)$ for $n$ players in this setting is given by

$$b^*(v) = v - \frac{\int_0^v F(x)^{n-1} dx}{F(v)^{n-1}} $$

We implement it here for calculating comparison metrics.

In [ ]:
import scipy.integrate as integrate
import warnings

common_dist = torch.distributions.normal.Normal(loc = valuation_mean, scale = valuation_std)

# TODO: investigate where everything is allocated. possibly move GPU vectors to CPU completely instead of shuffling around for integration?
def optimal_bid(valuation: torch.Tensor or np.ndarray or float) -> torch.Tensor:
    
    # For float and numpy --> convert to tensor
    if not isinstance(valuation, torch.Tensor):
        valuation = torch.tensor(valuation, dtype = torch.float)           
    # For float / 0d tensors --> unsqueeze to allow list comprehension below
    if valuation.dim() == 0:
        valuation.unsqueeze_(0)
    
    # shorthand notation for F^(n-1)
    Fpowered = lambda v: torch.pow(common_dist.cdf(v), n_players - 1)  
    
    # do the calculations
    numerator = torch.tensor(
            [integrate.quad(Fpowered, 0, v)[0] for v in valuation],
            device = valuation.device
        ).reshape(valuation.shape)                                 
    return valuation - numerator / Fpowered(valuation)

# equilibrium Strategy and environment in equilibirum
bneStrategy = ClosureStragegy(optimal_bid, parallel=n_processes_optimal_strategy)
# set up an environment populated by players that play the optimum
bne_env = AuctionEnvironment(mechanism,
                             agents = [strat_to_bidder(bneStrategy,
                                                       player_position= i,
                                                       batch_size = eval_batch_size,
                                                       cache_actions = True) 
                                       for i in range(n_players)
                                      ],
                             batch_size = eval_batch_size,
                             n_players=n_players,
                             strategy_to_player_closure = strat_to_bidder
                             )

with warnings.catch_warnings(): 
    warnings.simplefilter('ignore')
    # don't print scipy accuracy warnings
    bne_utility = bne_env.get_reward(bne_env.agents[0], draw_valuations = True)
print('Utility in BNE: \t\t{:.5f}'.format(bne_utility))
utility_vs_bne = bne_env.get_strategy_reward(model, player_position=0)
print('Model utility vs BNE: \t\t{:.5f}'.format(utility_vs_bne))
utility_learning_env = env.get_strategy_reward(model, player_position=0, draw_valuations = True)
print('Model utility in learning env:\t{:.5f}'.format(utility_learning_env))
  
    
# predefine points for plotting optimal curve to save cpu-bound integrations
v_opt = np.linspace(plot_xmin, plot_xmax, 100) # 100 points more than enough
b_opt = optimal_bid(v_opt).numpy()

def plot_bid_function(fig, v,b, writer=None, e=None, plot_points=plot_points):
    
    # subsample points and plot
    v = v.detach().cpu().numpy()[:plot_points]
    b= b.detach().cpu().numpy()[:plot_points]
    
    fig = plt.gcf()
    plt.cla()
    plt.xlim(plot_xmin, plot_xmax)
    plt.ylim(plot_ymin, plot_ymax)
    plt.plot(v,b, 'o', v_opt, b_opt, 'r--')
    #if is_ipython:
    #    display.clear_output(wait=True)
    display.display(plt.gcf())
    if writer:
        writer.add_figure('eval/bid_function', fig, e)  

Plot optimal bid to ensure appropriate boundaries have been chosen

In [ ]:
plot_bid_function(None, torch.tensor([0]),torch.tensor([0]))

## Set up training loop

## Training

In [ ]:
def log_once(writer, e):
    """Everything that should be logged only once on initialization."""
    writer.add_scalar('debug/total_model_parameters', n_parameters, e)
    writer.add_text('hyperparams/neural_net_spec', str(model), 0)    
    writer.add_scalar('debug/eval_batch_size', eval_batch_size, e)
    writer.add_graph(model, env.agents[0].valuations)   

def log_hyperparams(writer, e):
    writer.add_scalar('hyperparams/batch_size', batch_size, e)
    writer.add_scalar('hyperparams/learning_rate', learning_rate, e)
    writer.add_scalar('hyperparams/momentum', momentum, e)
    writer.add_scalar('hyperparams/sigma', sigma, e)
    writer.add_scalar('hyperparams/n_perturbations', n_perturbations, e)

def training_loop(e, writer):
    
    global overhead_mins, learning_rate
    
    if lr_decay and e % lr_decay_every == 0 and e > 0:
        learning_rate = learning_rate * lr_decay_factor
        log_hyperparams(writer, e)
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
    
    ### do in every iteration ###
    # save current params to calculate update norm
    prev_params = torch.nn.utils.parameters_to_vector(model.parameters())
    #update model
    utility = -optimizer.step()
    # calculate infinity-norm of update step
    new_params = torch.nn.utils.parameters_to_vector(model.parameters())
    update_norm = (new_params - prev_params).norm(float('inf'))    

    # calculate utility vs bne    
    utility_vs_bne = bne_env.get_reward(strat_to_bidder(model, batch_size = eval_batch_size), draw_valuations=False)
    epsilon_relative = 1 - utility_vs_bne / bne_utility
    epsilon_absolute = bne_utility - utility_vs_bne
    
    #log those things
    writer.add_scalar('eval/utility', utility, e)
    writer.add_scalar('debug/norm_parameter_update', update_norm, e)
    writer.add_scalar('eval/utility_vs_bne', utility_vs_bne, e)
    writer.add_scalar('eval/epsilon_relative', epsilon_relative, e)
    writer.add_scalar('debug/epsilon_absolute', epsilon_absolute, e) # debug because only interesting to see if numeric precision is a problem, otherwise same as relative but scaled.

    if e % plot_epoch == 0:
        start_time = timer()
        # plot current function output
        v = bidders[0].valuations
        b = bidders[0].get_action()

        #writer.add_graph(model, bidder.valuations) 

        print("Epoch {}: \tcurrent utility: {:.3f},\t utility vs BNE: {:.3f}, \tepsilon (abs/rel): ({:.5f}, {:.5f})".format(e, utility, utility_vs_bne, epsilon_absolute, epsilon_relative))
        plot_bid_function(fig, v,b,writer,e)
            
        elapsed = timer() - start_time
            
        overhead_mins = overhead_mins + elapsed/60
        writer.add_scalar('debug/overhead_mins', overhead_mins, e)
            
        print("Logging checkpoint took {:.2f}s.".format(elapsed))  

In [ ]:
#if not 'resume' in locals() or not resume:
#    e = 0
#    overhead_mins = 0

# setup logger
if os.name == 'nt': raise ValueError('The run_name may not contain : on Windows! (change datetime format to fix this)') 
run_name = time.strftime('%Y-%m-%d %a %H:%M')
if run_comment:
    run_name = run_name + ' - ' + str(run_comment)
logdir = os.path.join(root_path, 'notebooks', 'fpsb', str(n_players) + 'p', 'normal', 'symmetric', run_name)

e = 0
overhead_mins = 0

print(logdir)
fig = plt.figure()
torch.cuda.empty_cache()

with SummaryWriter(logdir, flush_secs=30) as writer:
    
    torch.cuda.empty_cache()
    log_once(writer, 0)
    log_hyperparams(writer, 0)    
    
    for e in range(e,e+epoch+1):
        training_loop(e, writer)     